<a href="https://colab.research.google.com/github/thim0o/Google-Colab-Peerflix-Server/blob/master/pf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Go to Runtime --> Run all (or ctrl+F9)


---

https://github.com/thim0o/Google-Colab-Peerflix-Server


# Install

In [0]:
#@markdown <h3>📝INSTALL NODEJS</h3>
import logging
import os

if os.path.exists("/tools/node/") == False:
  !apt install nodejs
  !npm i -g npm
else:
  !echo "nodejs Already installed"


In [0]:
#@markdown <h3>📝INSTALL PEERFLIX-SERVER</h3>
from IPython.display import HTML, clear_output
if os.path.exists("/tools/node/lib/node_modules/peerflix-server/") == False:
  !npm install -g peerflix-server
  clear_output(wait=True)
  !echo "succeeded"
else:
  !echo "peerflix-server Already installed"



# Configure

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝CREATE PEERFLIX CONFIG FILE</h3>
connections = 500 #@param {type:"integer"}
addTrackersFromGithub = True #@param {type:"boolean"}
#https://github.com/thim0o/Google-Colab-Peerflix-Server
# ============================= FORM ============================= #

import os
import logging
import json
import requests
from IPython.display import HTML, clear_output


#CREATE THE CONFIG
data = {}  

data['connections'] = connections
data['path'] = "/content/peerflix/" 
data['dht'] = True 
data['tracker'] = True

if addTrackersFromGithub:
  responseText = requests.get("https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_all.txt").text
  trackers = responseText.split("\n\n")
  data['trackers'] = trackers
  
  infoText = str(len(trackers)) + " Trackers loaded from github"
  
  !echo $infoText



# SAVE THE CONFIG
!mkdir /root/.config/peerflix-server/
configFile = open(r"/root/.config/peerflix-server/config.json", "w+")
json.dump(data, configFile)
configFile.close()



# PRINT THE CONFIG
f = open(r"/root/.config/peerflix-server/config.json", "r")
text = f.read()
f.close()



In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Enter which localhost port</h3>
port = 9001 #@param {type:"integer"}

#@markdown <h3>📝 Subdomain (Random if empty) (NOTE: this has to be unique!!!)</h3>
subdomain = "" #@param {type:"string"}
#https://github.com/thim0o/Google-Colab-Peerflix-Server
# ============================= FORM ============================= #


import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output
import logging

!pkill autossh

SuccessRun = widgets.Button(
    description='✔ Successfully',
    disabled=True,
    button_style='success'
)

UnsuccessfullyRun = widgets.Button(
    description='✘ Unsuccessfully',
    disabled=True,
    button_style='danger'
)

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

def AutoSSH(name,port):
  get_ipython().system_raw("autossh -l " + name + " -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:" + port + " ssh.localhost.run &")
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_ServerMT():
  if CheckProcess("autossh",Random_NumberMT) != True:
      AutoSSH(Random_NumberMT, str(port))

try:
  if subdomain == "":
    Random_NumberMT = RandomGenerator()
  else:
    Random_NumberMT = subdomain
  if os.path.isfile("/usr/bin/autossh") == False:
    !apt update -qq -y
    !apt install autossh -qq -y
    clear_output(wait=True)

  Start_ServerMT()
  display(SuccessRun)
  display(MakeButton("Recheck", Start_ServerMT))
  display(HTML("<h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_NumberMT + ".localhost.run\" target=\"_blank\">Website 1</a><br>" \
               "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_NumberMT + ".serveo.net\" target=\"_blank\">Website 2 (FAST)</a></h4>"))
except Exception as e:
  logging.error(e)
  clear_output(wait=True)
  display(UnsuccessfullyRun)

# Control

In [0]:
#@markdown <h3>📝STOP PEERFLIX</h3>
import psutil

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue
      

def stopPeerflixIfRunning():
  if CheckProcess("node", "/tools/node/bin/peerflix-server") == True:
      !pkill -f "node /tools/node/bin/peerflix-server"
      !echo "killed peerflix"
  else:
    !echo "peerflix was NOT running!!!"

stopPeerflixIfRunning()

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝START PEERFLIX</h3>
silent = True #@param {type:"boolean"}
killExisting = True #@param {type:"boolean"}
#https://github.com/thim0o/Google-Colab-Peerflix-Server
# ============================= FORM ============================= #


import os
import logging
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

if killExisting:
  !pkill -f "node /tools/node/bin/peerflix-server"


if os.path.exists("/tools/node/lib/node_modules/peerflix-server/") == False:
  logging.error("PEERFLIX NOT INSTALLED!!!")
  raise Exception


if silent:
  !nohup peerflix-server >& /dev/null &
  !echo "started silent"

else:
  !peerflix-server

try:
  display(HTML("<h1 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_NumberMT + ".localhost.run\" target=\"_blank\">Website 1</a><br>" \
                 "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_NumberMT + ".serveo.net\" target=\"_blank\">Website 2 (FAST)</a></h1>"))
except:
  !echo "Server running. Go click on the link above!"


# Tools

# <img src='https://minormole.github.io/RcloneLab/img/title_utility.png' height="45" alt="Utility"/>

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>Check VM's Status</h3>
Loop_Check = False #@param {type:"boolean"}
Loop_Interval = 15 #@param {type:"slider", min:1, max:15, step:1}
# ================================================================ #

import time
from IPython.display import clear_output
Loop = True
!uptime 

try:
  while Loop == True:
    clear_output(wait=True)
    !top -bcn1 -w512
    if Loop_Check == False:
      Loop = False
    else:
      time.sleep(Loop_Interval)
except:
  clear_output()

In [0]:
# ============================= FORM ============================= #
#@markdown <br><center><img src='https://minormole.github.io/RcloneLab/img/title_netdata.svg' height="100" alt="netdata"/></center>
#@markdown <center><h3>Real-time Server Monitoring</h3></center><br>
# ============================= FORM ============================= #

TornOnNetdata = False #@param {type:"boolean"}

# ============================= FORM ============================= #
# ================================================================ #

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output
import sys




SuccessRun = widgets.Button(
    description='✔ Successfully',
    disabled=True,
    button_style='success'
)

UnsuccessfullyRun = widgets.Button(
    description='✘ Unsuccessfully',
    disabled=True,
    button_style='danger'
)

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

def AutoSSH(name,port):
  get_ipython().system_raw("autossh -l " + name + " -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:" + port + " ssh.localhost.run &")
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_ServerMT():
  if CheckProcess("netdata", "") != True:
    !/usr/sbin/netdata
  if CheckProcess("autossh", "mt" + Random_NumberMT) != True:
      AutoSSH("mt" + Random_NumberMT, "19999")

      
      
if TornOnNetdata:


  try:
    try:
      Random_NumberMT
    except NameError:
      Random_NumberMT = RandomGenerator()
    if os.path.isfile("/usr/bin/autossh") == False:
      !apt update -qq -y
      !apt install autossh -qq -y
      clear_output(wait=True)
    if os.path.isfile("/usr/sbin/netdata") == False:
      !bash <(curl -Ss https://my-netdata.io/kickstart.sh) --dont-wait --dont-start-it
      clear_output(wait=True)
    Start_ServerMT()
    display(SuccessRun)
    display(MakeButton("Recheck", Start_ServerMT))
    display(HTML("<h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://mt" + Random_NumberMT + ".localhost.run\" target=\"_blank\">Website 1</a><br>" \
                 "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://mt" + Random_NumberMT + ".serveo.net\" target=\"_blank\">Website 2</a></h4>"))
  except:
    clear_output(wait=True)
    display(UnsuccessfullyRun)
else:
  !echo "netdata not turned on"